<a href="https://colab.research.google.com/github/AlirezaPNouri/Storytelling/blob/main/FinalExperimentalForPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The final version of storytelling algorithm to generate the experimental results for paper. 5/6/2022

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.1 MB/s 
     |████████████████████████████████| 880 kB 52.1 MB/s 
     |████████████████████████████████| 6.6 MB 77.1 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1b1997b3dcb04e5f4e169846d62fc522ee456f14514af5d89513f11649992c75
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
# needed packages
import requests
import numpy as np
import pandas as pd
import string
import pickle
import os
from numpy import median, array, exp
from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [3]:
#### Load the pre-traiend BERT model #####################
##########################################################
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [26]:
#constant variables
dataset_covid_link = 'https://raw.githubusercontent.com/AlirezaPNouri/Storytelling/main/datasets/short_parsed_covid_data.csv'
dataset_NYT_link = 'https://raw.githubusercontent.com/AlirezaPNouri/Storytelling/main/datasets/short_parsed_NYT_data.csv'
dataset_percentage = .6
MAX_DATA = 100
MAX_SENTENCE_LENGTH = 400
MIN_SENTENCE_LENGTH = 200
doc_list =[item for item in range(MAX_DATA)]
jump = 3 # jump size for gradient decent
MAX_FEATURE_SIZE = 20 # max size of the gradient descent vector
range_n_clusters = [2, 3, 4, 5, 6] #number of different clustering to compare the performance of the model over different clustering
threshold_cluster_overlapping = 0.7 # to consider two clsuters are similar
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'of', 'within']


In [5]:
# download the dataset as a zip file from the git repo
def dataset_downloader(str_):
  """
  This function download a dataset
  Arguments:
    str_: the name of the dataset. It can be covid or NYT
  Returns:
    df: a pandas dataframe
  """
  if str_ == 'covid':
    url = dataset_NYT_link
  elif str_ == 'NYT':
    url = dataset_covid_link
  else:
    print('dataset is not choose correctly!')

  print('Downloading dataset...')
  res = requests.get(url, allow_redirects=True)
  with open('small_dataset_NYT.csv','wb') as file:
      file.write(res.content)
  print('Download is done!')
  df = pd.read_csv("/content/small_dataset_NYT.csv", header= None, skiprows=1)
  if str_ == 'NYT':
    df.columns = ['id', 'title', 'content', 'publish_time', 'author']
  elif str_ == 'covid':
    df.columns = ['id', 'title', 'content', 'author', 'publish_time']
  
  df = df.dropna(how='any', axis=0)
  print('dataset size after removing non-value cells is {}'.format(df.shape))
  # reduce the size of dataset to dataset_percentage*dataset.shape
  df = df.sample(frac=dataset_percentage)
  print('The new size of dataset is {} and the columns are {}'.format(df.shape, df.columns.values ))
  # remove the content that has less than MIN_SENTENCE_LENGTH words 
  df = df[df['content'].apply(lambda x : len(x.split(' ')))>=MIN_SENTENCE_LENGTH]

  # keep the first 500 words in content
  df['content'] = df['content'].apply(lambda x : ' '.join(x.split(' ')[:MAX_SENTENCE_LENGTH]))
  return df


In [6]:
def special_char_remover(str_):
  """
  This function removes any special characters from a text
  Arguments:
    str_: a string
  Returns:
    A string
  """
  return ' '.join(''.join(w for w in m if w.isalnum()) for m in str_.split(' '))

In [7]:
### create embedding vector for each token in each document
def bert_embedding_generator(df_):
  """
  Generate the embedding for a document
  Arguments:
    df_: a pandas dataframe 
  Returns:
    Embedding_dict: a dictionary of document indices and [embedding vectors, content]
  """
  Embedding_dict = dict()
  print('Start to generate the embedding vector for documents')
  c = 0
  for index, row in df_.iterrows():
    tmp_e = model(tf.constant(tokenizer.encode(row['content']))[None, :])[0][0]
    tmp_d = tokenizer.tokenize(row['content'])
    temp_dict = dict()
    temp_dict['cls'] = tmp_e[0]
    for emb, tok in zip(tmp_e[1:-1], tmp_d):
      temp_dict[tok] = emb
    temp_dict['sep'] = tmp_e[-1]
    Embedding_dict[row['id']] = temp_dict
    c +=1
  print('Embedding is done for {} documents.'.format(c))
  return Embedding_dict

In [8]:
#### Create a list of dataframe out of the dataset. Each dataframe belongs to a timestamp #############
#######################################################################################################
# use 10 days as time interval
def timestamps_generator(df_):
  """
  This function collect data related to each timestamp separately in a dataframe
  Arguments:
    df_: a pandas dataframe
  Returns:
    df_list: a list of all timestamps data [df_1, df_2, ...]
  """
  time_stamp = set()
  df_list= list()
  for index, row in df_.iterrows():
    time_stamp.add(row['publish_time'][0:3])
  for time_interval in time_stamp:
    new_df = df_[df_['publish_time'].str.slice(0, 3)== time_interval]
    if new_df.shape[0]>5: #minimum number of doc in a timestamp
      df_list.append(new_df)
  return df_list

In [9]:
def time_fixer(str_):
  """
  This function convert a date from x/x/xxxx to 0x0xxxxx
  Arguments:
    str_: a date in a string format
  Returns:
    a string without any / and all months and days are in two digit
  """
  t_slot = str_.split('/')
  t_slot[0] = t_slot[0] if len(t_slot[0]) == 2 else '0'+t_slot[0]
  t_slot[1] = t_slot[1] if len(t_slot[1]) == 2 else '0'+t_slot[1]
  return ''+t_slot[0]+t_slot[1]+t_slot[2]

In [12]:
def find_similarity(Embed_):
  """
  This function calculate the cosine similarity between each pair of words in a document and sort them as a new vector
  Arguments:
    Embed_: a dictionary of all doc and embedding seperated into their timestamps {timestamp1 {doc1{token1: embedding, token2: embedding}}}
  Returns:
    neighbor_ts_doc: a dictionary of all timestamps, docs and the neighbor similarity for each word
  """
  neighbor_ts_doc = dict()
  for ts_ in Embed_.keys():
    temp = Embed_[ts_]
    final_list = dict()
    for doc in temp.keys():
      neighbor_dict = dict()
      track_dict = dict()
      for f_word, f_emb in temp[doc].items():
        temp_list = list()
        for s_word, s_emb in temp[doc].items():
          if (s_word, f_word) in track_dict.keys():
            temp_list.append(track_dict[(s_word, f_word)])
          else:
            track_dict[(f_word, s_word)] = cosine_similarity([f_emb],[s_emb])[0][0]
            temp_list.append(track_dict[(f_word, s_word)])
        neighbor_dict[f_word] =sorted(temp_list, reverse=True)[1:] # ignore the cosine similarity between a word and itself
      final_list[doc] = neighbor_dict
      print('Document {} is done!'.format(doc))
    neighbor_ts_doc[ts_] = final_list
  return neighbor_ts_doc

In [27]:

#sigmoid function
def amplifier_function(vec_):
  """
  This function receives a vector and applies the curve below to reward the first part of the vector and to penalize the last part of the vector
  Arguments:
    vec_: a vector
  Results:
    g: a vector
  """
  z = array(vec_)
  z = z/np.linalg.norm(z)
  g = 1 / (1 + exp(-z))
  return g

In [28]:
def gradient_calculator(ns_dict, n_step, MAX_FEATURE_SIZE):
  """
  This function receives all the similarity neighbor for each word in all documents and apply gradient descent over them by the step
  Arguments:
    ns_dict: a dictinary of all neighbor similarity for all tokens
    n_step: the jump value for gradient discent
    MAX_FEATURE_SIZE: the maximum size for each feature victor
  Returns:
    final_dict: a dictionary of all gradient discent of tokens neighbor similarity
  """
  final_dict = dict()
  for ts_ in ns_dict.keys():
    gradient_descent_neighbors = dict()
    word_neighbor_similarity = ns_dict[ts_]
    for doc in word_neighbor_similarity.keys():
      temp_dict = dict()
      for word, embedding in word_neighbor_similarity[doc].items():
        vec_size = len(embedding)
        vec_size -= vec_size%n_step
        embedding = embedding[0:vec_size]
        new_list = [embedding[x:x+n_step] for x in range(0, vec_size, n_step)]
        tmp_list = list()
        for ele in new_list:
          m1,b1 = np.polyfit(np.arange(0, n_step), ele, 1)
          tmp_list.append(m1)
        temp_dict[word] = amplifier_function(tmp_list[0:MAX_FEATURE_SIZE])
      gradient_descent_neighbors[doc] = temp_dict
      print('Document {} is done!'.format(doc))
    final_dict[ts_] = gradient_descent_neighbors
  return final_dict

In [31]:

# clustering the tokens in each document. It creates differnt number of clustering

def cluster_maker(final_vec_):
  """
  This function generates clusters based on featur vectors
  Arguments:
    final_vec_: a dictionary of all gradient discent of tokens neighbor similarity
  Returns:
    total_clustering: a dictionary contains timestamps and clusters in each timestamp 
    total_centroid: a dictionart contains timestamps and centroids for each clusters
  """
  total_clustering = dict()
  total_centroid = dict()
  for ts_ in final_vec_.keys():
    gradient_descent_neighbors = final_vec_[ts_]
    all_clusters = dict()
    all_centroid = dict()
    for doc, featurs in gradient_descent_neighbors.items():
      clusters_per_doc = dict()
      centroids_per_doc = dict()
      for n_clusters in range_n_clusters:
        clusterer = KMeans(n_clusters=n_clusters, random_state=10, max_iter=100,)
        clusters_per_doc[n_clusters] = clusterer.fit_predict(list(featurs.values()))
        centroids_per_doc[n_clusters] = clusterer.cluster_centers_
      all_clusters[doc] = clusters_per_doc
      all_centroid[doc] = centroids_per_doc
      print('Document {} is done!'.format(doc))
    total_clustering[ts_] = all_clusters
    total_centroid[ts_] = all_centroid
  return total_clustering, total_centroid
    

In [36]:
def sorted_clustering(all_gradient_descent_neighbors,all_clusters_dict, all_centroids_dict):
  final_clustering_dict = dict()
  for ts_ in all_clusters_dict.keys():
    gradient_descent_neighbors = all_gradient_descent_neighbors[ts_]
    all_centroid = all_centroids_dict[ts_]
    all_clusters = all_clusters_dict[ts_]
    #create a total clusters sorted dictionary
    ## the structure of total_docs_clusters  is {doc0: {number_of_clusters { cluster_number: {token0: closeness_to_centroid, token1: closeness_to_centroid ...} }}}
    total_docs_clusters = dict()
    for doc in all_clusters.keys():
      temp_cluster = all_clusters[doc]
      dic_for_clustering = dict()
      for n_cluster in range_n_clusters:
        temp_dict = dict()
        for la in range(n_cluster):
          temp_list = dict()
          for label_, value_ in zip(temp_cluster[n_cluster],gradient_descent_neighbors[doc].items()):
            if label_ == la:
              temp_list[value_[0]] = np.sum(np.square(value_[1] - all_centroid[doc][n_cluster][la]))
          temp_dict[la]=temp_list
        dic_for_clustering[n_cluster] = temp_dict
      total_docs_clusters[doc] = dic_for_clustering
      print('Document {} is done!'.format(doc))
    final_clustering_dict[ts_] = total_docs_clusters
  return final_clustering_dict

In [39]:
def normalize_vec(vec, min_, max_):
  """
  # this function will receive a vector and the min and max of that vector and will return the normalized vec 
  """
  range_val = (max_ - min_)+0.0000001
  vec = [item/range_val for item in vec]
  min_vec = min(vec)
  vec = [item - min_vec for item in vec]
  return vec



def normalize_value(item, min_, max_):
  """
  # this function will receive a value of a vector, the min and the max of the vector and returns the normalized value
  """
  range_val = (max_ - min_)+0.0000001
  vec = (item-min_)/range_val
  return vec

In [43]:
def normalizer(total_docs_clustering_):
  """
    normalize the similarity in each cluster. It goes and find the min and max values in each clustering and make those similarity between [0,1] (the similarity here is euclidean distance between each token and the centroid)

  """
  all_total_docs_clusters = dict()
  for ts_ in total_docs_clustering_.keys():
    total_docs_clusters = total_docs_clustering_[ts_]
    for doc in total_docs_clusters.keys():
      temp_doc = total_docs_clusters[doc]
      for cluster_rank in temp_doc.keys():
        temp_clustering = temp_doc[cluster_rank]
        min_sim = 1
        max_sim = 0
        for ind in temp_clustering.keys():
          for ele, val_ in temp_clustering[ind].items():
            min_sim = min_sim if min_sim<val_ else val_
            max_sim = max_sim if val_<max_sim else val_ 

        for ins in temp_clustering.keys():
          for el in total_docs_clusters[doc][cluster_rank][ins].keys():
            total_docs_clusters[doc][cluster_rank][ins][el] = normalize_value(total_docs_clusters[doc][cluster_rank][ins][el], min_sim, max_sim)
      print('Document {} is done!'.format(doc))
    all_total_docs_clusters[ts_] = total_docs_clusters
  return all_total_docs_clusters

In [45]:
def stopword_remover(all_dict_):
  final_dict_w_stopword = dict()
  for ts_ in all_dict_.keys():
    total_docs_clusters = all_dict_[ts_]
    #remove stopwords
    total_docs_clusters_wo_stopwords = dict()
    for doc in total_docs_clusters.keys():
      temp_doc = {}
      for cluster_rank in total_docs_clusters[doc].keys():
        temp_clustering = {}
        for ins in total_docs_clusters[doc][cluster_rank].keys():
          temp_cluster = {}
          for items in total_docs_clusters[doc][cluster_rank][ins].items():
            if items[0] not in stopwords:
              temp_cluster[items[0]] = items[1]
          temp_clustering[ins] = temp_cluster
        temp_doc[cluster_rank] = temp_clustering
      total_docs_clusters_wo_stopwords[doc] = temp_doc
    final_dict_w_stopword[ts_] = total_docs_clusters_wo_stopwords
  return final_dict_w_stopword

In [47]:
# Jaccard index function
def jaccard_similarity(text1_list, text2_list):
  a = set(text1_list)
  b = set(text2_list)
  return len(a.intersection(b))/len(a.union(b))

In [48]:
def word_group_generator(all_clusters_):
  final_total_doc_word_group = dict()
  for ts_ in all_clusters_.keys():
    total_docs_clusters_wo_stopwords = all_clusters_[ts_]
    total_doc_word_groups = dict()
    for doc in total_docs_clusters_wo_stopwords.keys():
      docs_clusters = total_docs_clusters_wo_stopwords[doc]
      # print(docs_clusters)
      temp_list_word_groups = list()
      for cluster_num in list(docs_clusters.keys())[:-1]:
        for cluster in range(cluster_num):
          for next_cluster in range(cluster_num+1):
            cl1 = list()
            cl1_w_sim = dict()
            cl2 = list()
            cl2_w_sim = dict()
            joint_list = list()
            for item in docs_clusters[cluster_num][cluster].items():
              cl1.append(item[0])
              cl1_w_sim[item[0]] =  item[1]
            for another_item in docs_clusters[cluster_num+1][next_cluster].items():
              cl2.append(another_item[0])
              cl2_w_sim[another_item[0]] = another_item[1]
            # print(' ({}, {}, {}) --> '.format(cluster_num, cluster, next_cluster), jaccard_similarity(cl1, cl2))
            jac_val = jaccard_similarity(cl1, cl2)
            if jac_val > threshold_cluster_overlapping:
              jnt_list = list(set(cl1).intersection(cl2))
              dict_vocab = dict()
              for el in jnt_list:
                dict_vocab[el] = max(cl1_w_sim[el], cl2_w_sim[el])

              temp_list_word_groups.append([dict_vocab ,jac_val])
      

      total_doc_word_groups[doc] = temp_list_word_groups
      # the structure of total_doc_word_group is {doc0: [ [ {word1: normalized_normalized_distance_to_centroid, ....}, similarity between two clusters] [{word1: normalized_distance_to_centroid, ....}, similarity_between_two_clusters]... ], doc1: ...}
    final_total_doc_word_group[ts_] = total_doc_word_groups
  return final_total_doc_word_group

In [59]:
def distance_2_freq(all_clusters_):
  """
  change the distance to centroid to the frequency [0, 1] --> [10, 1] 
  """
  final_dict = dict()
  for ts_ in all_clusters_.keys():
    total_doc_word_groups_ = all_clusters_[ts_]
    #change the distance to centroid to the frequency [0, 1] --> [10, 1] 
    for doc in total_doc_word_groups_.keys():
      temp_list = list()
      for word_list in total_doc_word_groups_[doc]:
        temp_dict = dict()
        for ele in word_list[0].keys():
          temp_dict[ele] =int(9* (1-word_list[0][ele]))+1
        temp_list.append(temp_dict)
      total_doc_word_groups_[doc] = temp_list
    final_dict[ts_] = total_doc_word_groups_
  return final_dict
    # the new structure of total_doc_word_groups_ is {doc0: [{groupOnewords}, grouptwowords], doc1}   , grouponewords = {word1:frequency, word2:ferquency}

In [83]:
def bert_classifier_generator(total_doc_word_embedding):
  """
  doc_matrix is a dictionary of each doc and other doc similarity to this doc based on BERT cls vector             doc_embedding ={doc0: {1: int(9*euclidian_distance +1 ), 2:int(9*euclidian_distance +1 )}, doc1}  

  """
  total_dict_ = dict()
  for ts_ in total_doc_word_embedding.keys():
    doc_word_embedding = total_doc_word_embedding[ts_]
    doc_embedding = dict()
    for first_doc in doc_word_embedding.keys():
      temp_dict = dict()
      for second_doc in doc_word_embedding.keys(): 
        if first_doc != second_doc:
          # temp_dict[second_doc] =int(9*np.square(sum(doc_word_embedding[first_doc]['cls']-doc_word_embedding[second_doc]['cls']))+1)
          temp_dict[second_doc] =int(9*np.linalg.norm(doc_word_embedding[first_doc]['cls']-doc_word_embedding[second_doc]['cls'])+1)
      doc_embedding[first_doc] = temp_dict
      print('Document {} is done!'.format(first_doc) )
    total_dict_[ts_] = doc_embedding
  return total_dict_
    


In [77]:
def similarity_documents(all_dict_):
  """
    calculate the similarity between two documents by using jaccard index of all word groups in each cluster by considering the frequency of those words
    the structure of total_clustering_w_frequency is {doc0:[[word_groups0],[word_groups1], ...], doc1: }
  """
  total_dict_ = dict()
  for ts_ in all_dict_.keys():
    total_doc_word_groups_ = all_dict_[ts_]
    total_clustering_w_frequency = dict()
    for doc in total_doc_word_groups_.keys():
      list_word_cloud = list()
      for word_group in total_doc_word_groups_[doc]:
        temp_list = list()
        for word in word_group.items():
          for ind in range(1, word[1]+1):
            temp_list.append(word[0])
        list_word_cloud.append(temp_list)
      total_clustering_w_frequency[doc] = list_word_cloud
      print('document {} is done!'.format(doc))
    total_dict_[ts_] = total_clustering_w_frequency
  return total_dict_


In [85]:
def similar_doc(doc_one, doc_two):
  """
  #find similar documents
  """
  list_sim = list()
  for clu_1 in doc_one:
    total_jac = 0
    for clu_2 in doc_two:
      jac_val = jaccard_similarity(clu_1, clu_2)
      total_jac += jac_val
    # total_jac /= len(doc_two)
    list_sim.append(total_jac)
  return sum(list_sim)
  # return sum(list_sim)/(len(doc_one)*len(doc_two))   #with normalizing based on the number of word groups

In [98]:
def weight_matrix_generator(all_dict_):
  final_dict_ = dict()
  for ts_ in all_dict_.keys():
    total_clustering_w_frequency_ = all_dict_[ts_]
    #create a matrix of document similarity based on the jaccard index of the similar words in each word group in each document
    doc_matrix=dict()
    for first_doc in total_clustering_w_frequency_.keys():
      temp_dict = dict()
      for second_doc in total_clustering_w_frequency_.keys():
        if first_doc != second_doc:
          temp_dict[second_doc] = similar_doc(total_clustering_w_frequency_[first_doc], total_clustering_w_frequency_[second_doc])
      doc_matrix[first_doc] = temp_dict
      print('Document {} is done'.format(first_doc))
    
    #normalize the weight and invert it. if an index is high that means they are similar so the weight must be low

    for doc in doc_matrix.keys():
      min_ = 10000
      max_ = -10000
      for ele in doc_matrix[doc].items():
        min_ = min_ if min_<ele[1] else ele[1]
        max_ = max_ if max_> ele[1] else ele[1]
      for ele in doc_matrix[doc].keys():
        doc_matrix[doc][ele] = int(99*(1-normalize_value(doc_matrix[doc][ele], min_, max_))+1)

    final_dict_[ts_]= doc_matrix
  return final_dict_


In [119]:
# A*Search algorithm

from collections import deque

class Graph:

    def __init__(self,heuristic, adjacency_list, start_node, stop_node):
        self.adjacency_list = adjacency_list
        self.H = dict()
        self.start_node = start_node
        self.stop_node = stop_node
        self.heuristic = heuristic
        for ele in heuristic[stop_node]:
          self.H[ele[0]] = ele[1]
        # self.H = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1}
      
    # def get_neighbors(self, v):
    #     return self.adjacency_list[v]
    # # heuristic function with equal values for all nodes
    
    # def h(self, n):
    #   # H = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1}
    #   return self.H[n]
    def a_star_algorithm(self):

      # open_list is a list of nodes which have been visited, but who's neighbors
      # haven't all been inspected, starts off with the start node
      # closed_list is a list of nodes which have been visited
      # and who's neighbors have been inspected
      open_list = set([self.start_node])
      closed_list = set([])
      # g contains current distances from start_node to all other nodes
      # the default value (if it's not found in the map) is +infinity
      g = {}
      g[self.start_node] = 0
      # parents contains an adjacency map of all nodes
      parents = {}
      parents[self.start_node] = self.start_node
      while len(open_list) > 0:
          n = None
          # find a node with the lowest value of f() - evaluation function
          for v in open_list:
              if n == None or g[v] + self.H[v] < g[n] + self.H[n]:
              # if n == None or g[v] + self.H[v] < g[n] + self.H[n]:
                  n = v;
          if n == None:
              print('Path does not exist!')
              return None
          # if the current node is the stop_node
          # then we begin reconstructin the path from it to the start_node
          if n == self.stop_node:
              reconst_path = []
              while parents[n] != n:
                  reconst_path.append(n)
                  n = parents[n]
              reconst_path.append(self.start_node)
              reconst_path.reverse()
              print('Path found: {}'.format(reconst_path))
              return reconst_path
          # for all neighbors of the current node do
          for (m, weight) in self.adjacency_list[n]:
              # if the current node isn't in both open_list and closed_list
              # add it to open_list and note n as it's parent
              avr = 0
              sam = 0
              for t in self.heuristic[n]: 
                  avr += t[1]
                  if t[0] == m:
                    sam = t[1]
              if sam > avr/2:
                  continue
              if m not in open_list and m not in closed_list:
                  open_list.add(m)
                  parents[m] = n
                  g[m] = g[n] + weight
              # otherwise, check if it's quicker to first visit n, then m
              # and if it is, update parent data and g data
              # and if the node was in the closed_list, move it to open_list
              else:
                  if g[m] > g[n] + weight:
                      g[m] = g[n] + weight
                      parents[m] = n
                      if m in closed_list:
                          closed_list.remove(m)
                          open_list.add(m)
          # remove n from the open_list, and add it to closed_list
          # because all of his neighbors were inspected
          open_list.remove(n)
          closed_list.add(n)

      print('Path does not exist!')
      return None

In [117]:
def a_search(all_embedding, all_weight):
  result_dict = dict()
  for ts_ in all_embedding.keys():
    doc_embedding_ = all_embedding[ts_]
    doc_matrix_ = all_weight[ts_]
    start_ = list(doc_embedding_.keys())[0]
    end_ = list(doc_embedding_.keys())[-1]
    #create the adjacancy matrix from the euclidean distanse of the doc embedding
    #heuristic

    heuristic = dict()
    for first_doc in doc_embedding_.keys():
      temp_list = list()
      for second_doc in doc_embedding_.keys():
        if first_doc != second_doc:
          temp_list.append((second_doc, doc_embedding_[first_doc][second_doc]))
        else:
          temp_list.append((second_doc, 0))
      heuristic[first_doc] = temp_list
    # print('heuristic is ', heuristic)

    # the weights
    doc_weights = dict()
    for first_doc in doc_matrix_.keys():
      temp_dict = list()
      for second_doc in doc_matrix_.keys():
        if first_doc != second_doc:
          temp_dict.append((second_doc, doc_matrix_[first_doc][second_doc]))
      doc_weights[first_doc] = temp_dict
    # print('weights is ', doc_weights)
    graph1 = Graph(heuristic, doc_weights, start_, end_)
    result_dict[ts_] = graph1.a_star_algorithm()
    print('Timestamp {} is done!'.format(ts_))
  return result_dict

In [ ]:
df = dataset_downloader('NYT')
df['content'] = df['content'].apply( lambda x: special_char_remover(x)) # remove special characters
df['publish_time'] = df['publish_time'].apply(lambda x: time_fixer(x))
list_of_dfs = timestamps_generator(df)
Embed_dict = dict() # {timestamp1 {doc1{(number_of_tokens+1, 768)}, doc2{}}, timestamp2{}}
for ts, data_ in zip(range(len(list_of_dfs)), list_of_dfs):
   Embed_dict[ts] = bert_embedding_generator(data_)

# pickle.dump(Embed_dict, open('Embed_dict.p', 'wb')) # store embeddings
# pickle.dump(df, open('list_of_dfs.p', 'wb')) # store list_of_dfs

In [11]:
# Embed_dict = pickle.load(open('Embed_dict.p', 'rb')) #restore the embeddings
# list_of_dfs = pickle.load(open('list_of_dfs.p', 'rb')) #restore the list_of_dfs
# Getting all memory using os.popen()
total_memory, used_memory, free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])
print("RAM memory: used {} GB, total: {} GB".format(used_memory/1000,total_memory/1000))

RAM memory: used 2.719 GB, total: 26.066 GB


In [ ]:
final_vector = find_similarity(Embed_dict)

In [16]:
total_memory, used_memory, free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])
print("RAM memory: used {} GB, total: {} GB".format(used_memory/1000,total_memory/1000))

RAM memory: used 6.15 GB, total: 26.066 GB


In [17]:
pickle.dump(final_vector, open('final_vector.p', 'wb')) # store embeddings


In [ ]:
final_vector = gradient_calculator(final_vector, jump, MAX_FEATURE_SIZE)

In [ ]:
total_clustering, total_centroid = cluster_maker(final_vector)

In [ ]:
total_docs_clustering = sorted_clustering(final_vector, total_clustering, total_centroid)

In [42]:
print(total_docs_clustering[0][51028][2])

{0: {'cls': 0.004711008411068573, 'background': 0.0010550273578880027, 'strategic': 0.007309265694515655, 'healthcare': 0.0018460560831093922, 'purchasing': 0.0014844878488452008, 'sh': 0.003355389264434657, '##p': 0.0035951722917812886, 'critical': 0.0017929840274206773, 'function': 0.004246928153317895, 'of': 0.0026715235921559843, 'health': 0.0033152905945247266, 'financing': 0.0028607825431841167, 'enhance': 0.0063741675701260835, '##s': 0.0015720828463767853, 'the': 0.008215276526541675, 'optimal': 0.0023443350600126046, 'attain': 0.0026167174845473707, 'efficient': 0.0034675079423903774, 'use': 0.0033555449241477447, 'financial': 0.002053317923355656, 'resources': 0.004260044981121499, 'countries': 0.0019122895470554408, 'committed': 0.0012396356103428386, 'to': 0.0019118577660713093, 'coverage': 0.004936821992269138, 'uh': 0.0034327358454621553, '##c': 0.00309053799373629, 'made': 0.005482730406634464, 'progress': 0.005286163462815443, 'in': 0.00841706388123472, 'their': 0.00260

In [ ]:
normalized_total_docs_clusters = normalizer(total_docs_clustering)

In [46]:
total_docs_clusters_wo_stopwords = stopword_remover(normalized_total_docs_clusters)

In [57]:
total_doc_word_groups = word_group_generator(total_docs_clusters_wo_stopwords)

In [58]:
print(total_doc_word_groups[0][51028])

[[{'##yst': 0.36831145298791745, 'comparing': 0.4795174628370016, 'arrangements': 0.9999962099289416, 'analytical': 0.6831048765418425, 'appropriate': 0.3577190892207038, 'budget': 0.5525435649555459, 'processes': 0.4783618117745757, 'however': 0.5077028099872957, 'framework': 0.3014662934168053, 'benefit': 0.24853524249337935, 'insurance': 0.35950260001936657, 'improve': 0.449623334966023, 'benefits': 0.1986632764053262, 'performance': 0.2121375350172303, 'relevant': 0.10181550373451285, '##oh': 0.3647818368554993, '##a': 0.1756199043659348, 'universal': 0.40508167699707487, 'service': 0.1848905052556623, 'public': 0.35313803054420834, '##ing': 0.6593915245318083, '##nched': 0.3584618759433159, '##tre': 0.556537450341443, 'population': 0.44474738869255664, 'efficiency': 0.5653915966945361, 'based': 0.3418104576760097, 'reforms': 0.37375893634090673, 'poor': 0.3948293532406119, 'external': 0.27148833136374123, 'reflects': 0.2567194588550904, 'integrated': 0.30131726220049243, 'limited'

In [60]:
total_doc_word_groups = distance_2_freq(total_doc_word_groups)

In [61]:
print(total_doc_word_groups[0][51028])

[{'##yst': 6, 'comparing': 5, 'arrangements': 1, 'analytical': 3, 'appropriate': 6, 'budget': 5, 'processes': 5, 'however': 5, 'framework': 7, 'benefit': 7, 'insurance': 6, 'improve': 5, 'benefits': 8, 'performance': 8, 'relevant': 9, '##oh': 6, '##a': 8, 'universal': 6, 'service': 8, 'public': 6, '##ing': 4, '##nched': 6, '##tre': 4, 'population': 5, 'efficiency': 4, 'based': 6, 'reforms': 6, 'poor': 6, 'external': 7, 'reflects': 7, 'integrated': 7, 'limited': 6, 'actions': 8, 'factors': 7, 'goals': 5, 'system': 5, '##base': 8, 'major': 6}, {'##ider': 9, 'governments': 9, '##makers': 9, 'characterized': 9, 'making': 9, 'quality': 9, 'remains': 9, 'represents': 9, 'established': 9, 'im': 9, 'conclusion': 9, 'organization': 9, 'capacity': 8, 'programme': 9, 'split': 9, 'track': 9, 'countries': 9, 'policy': 9, 'res': 9, 'purchase': 9, 'optimal': 9, 'managers': 9, 'documents': 9, 'derived': 9, 'use': 9, 'collected': 7, 'accountability': 9, 'undertaken': 9, 'committed': 9, 'tax': 9, 'whose

In [ ]:
doc_embedding = bert_classifier_generator(Embed_dict)

In [73]:
print(doc_embedding[0][51028])

{531772: 81, 494379: 91, 119162: 94, 777456: 93, 67364: 95, 115108: 102, 767976: 108, 681624: 116, 107753: 89, 642286: 90, 51121: 96, 660610: 107, 614475: 85, 668878: 104, 56204: 108, 671324: 95, 550912: 97, 712161: 105, 620728: 92, 51273: 98, 165446: 101, 51830: 80, 51152: 111, 765722: 95, 504567: 95, 71913: 95, 51292: 112, 727098: 103, 490850: 79, 643986: 109, 50948: 103, 50984: 96, 679972: 105, 423221: 102, 568890: 104, 198222: 104, 166870: 94, 681452: 100, 138692: 91, 131518: 108, 686852: 106, 125596: 102, 201223: 93, 757007: 92, 762377: 109, 590137: 107, 524146: 102, 590558: 99, 658628: 85, 85977: 98, 601377: 96, 656702: 98, 109055: 86, 670676: 92, 502546: 101, 24206: 104, 724407: 86, 135984: 100, 506967: 105, 667553: 95, 572444: 103, 56393: 92, 248172: 106, 204896: 104, 492195: 110, 51167: 100, 94167: 102, 670195: 107, 567310: 110, 93986: 89, 98538: 94, 645869: 89, 605687: 96, 646708: 96, 51122: 110, 701508: 97, 700950: 107, 59997: 98, 535040: 88, 618324: 101, 492444: 98, 748335:

In [ ]:
total_clustering_w_frequency = similarity_documents(total_doc_word_groups)

In [79]:
print(total_clustering_w_frequency[0][51028])

[['##yst', '##yst', '##yst', '##yst', '##yst', '##yst', 'comparing', 'comparing', 'comparing', 'comparing', 'comparing', 'arrangements', 'analytical', 'analytical', 'analytical', 'appropriate', 'appropriate', 'appropriate', 'appropriate', 'appropriate', 'appropriate', 'budget', 'budget', 'budget', 'budget', 'budget', 'processes', 'processes', 'processes', 'processes', 'processes', 'however', 'however', 'however', 'however', 'however', 'framework', 'framework', 'framework', 'framework', 'framework', 'framework', 'framework', 'benefit', 'benefit', 'benefit', 'benefit', 'benefit', 'benefit', 'benefit', 'insurance', 'insurance', 'insurance', 'insurance', 'insurance', 'insurance', 'improve', 'improve', 'improve', 'improve', 'improve', 'benefits', 'benefits', 'benefits', 'benefits', 'benefits', 'benefits', 'benefits', 'benefits', 'performance', 'performance', 'performance', 'performance', 'performance', 'performance', 'performance', 'performance', 'relevant', 'relevant', 'relevant', 'relevan

In [86]:
print(similar_doc(total_clustering_w_frequency[0][51028], total_clustering_w_frequency[0][119162]))

0.6385326594436981


In [ ]:
doc_matrix_weight = weight_matrix_generator(total_clustering_w_frequency)

In [102]:

print(doc_matrix_weight[0][51028])
print(doc_embedding[0][51028])

{531772: 76, 494379: 4, 119162: 72, 777456: 69, 67364: 77, 115108: 64, 767976: 56, 681624: 61, 107753: 88, 642286: 56, 51121: 90, 660610: 70, 614475: 35, 668878: 68, 56204: 73, 671324: 61, 550912: 82, 712161: 63, 620728: 82, 51273: 50, 165446: 49, 51830: 47, 51152: 59, 765722: 77, 504567: 79, 71913: 81, 51292: 60, 727098: 48, 490850: 68, 643986: 74, 50948: 94, 50984: 79, 679972: 70, 423221: 78, 568890: 76, 198222: 67, 166870: 80, 681452: 36, 138692: 38, 131518: 84, 686852: 54, 125596: 66, 201223: 76, 757007: 57, 762377: 74, 590137: 59, 524146: 61, 590558: 74, 658628: 32, 85977: 92, 601377: 79, 656702: 71, 109055: 81, 670676: 28, 502546: 68, 24206: 67, 724407: 58, 135984: 89, 506967: 84, 667553: 66, 572444: 65, 56393: 58, 248172: 47, 204896: 75, 492195: 82, 51167: 69, 94167: 63, 670195: 72, 567310: 54, 93986: 79, 98538: 65, 645869: 51, 605687: 72, 646708: 66, 51122: 84, 701508: 82, 700950: 73, 59997: 61, 535040: 48, 618324: 60, 492444: 75, 748335: 43, 703004: 73, 582071: 81, 754166: 73,

In [109]:
print(list(doc_embedding.keys())[-1])

3


In [124]:
emb_ = [doc_embedding, doc_matrix_weight]
pickle.dump(emb_,open('emb_.p', 'wb'))

In [122]:
final_path = a_search(doc_embedding, doc_matrix_weight)

Path found: [51028, 51244]
Timestamp 0 is done!
Path found: [576786, 504434]
Timestamp 1 is done!
Path found: [652874, 663267]
Timestamp 2 is done!
Path found: [715517, 553817]
Timestamp 3 is done!
